# Read all images from local

In [59]:
import os
import glob  # 查找文件
from tensorflow import keras
import tensorflow as tf
import numpy as np
from pandas import Series, DataFrame
from six.moves import range
#加载数据
import os
from PIL import Image
import numpy as np
#读取文件夹train下的42000张图片，图片为彩色图，所以为3通道，
#如果是将彩色图作为输入,图像大小224*224
def load_data():
    imgs = os.listdir("/Users/apple/Desktop/all/train/")
    num = 2000
    data = np.empty((num,224,224,3),dtype="float32")
    label = np.empty((num,))
    time  = 0;
    for i in range(len(imgs)):
        if time > 1999:
            break;
        if imgs[i].split('.')[0] == 'cat':
            img = tf.keras.preprocessing.image.load_img("/Users/apple/Desktop/all/train/" + imgs[i],target_size=(224,224))
            arr = tf.keras.preprocessing.image.img_to_array(img)
            arr.resize((224,224,3))
            data[time, :, :, :] = arr
            label[time] = 0
            time = time + 1
        if imgs[i].split('.')[0] == 'dog':
            img = tf.keras.preprocessing.image.load_img("/Users/apple/Desktop/all/train/" + imgs[i],target_size=(224,224))
            arr = tf.keras.preprocessing.image.img_to_array(img)
            arr.resize((224, 224, 3))
            data[time, :, :, :] = arr
            label[time] = 1
            time = time+1
    return data,label
data,label = load_data()
print(data.shape)
train_data = data[:1800]
train_labels = label[:1800]
validation_data = data[1800:]
validation_labels = label[1800:]

(2000, 224, 224, 3)


In [60]:
train_data = train_data / 255.0

validation_data = validation_data / 255.0


In [61]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_data[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

# Create a new model by Keras

Create a new model. There are two layer for Convolution and one for Flatten

In [68]:
from tensorflow import keras
model = keras.models.Sequential()
#第一个卷积层，4个卷积核，每个卷积核大小5*5。
#激活函数用tanh
#还可以在model.add(Activation('tanh'))后加上dropout的技巧: model.add(Dropout(0.5))
model.add(keras.layers.Convolution2D(4, 5, 5,input_shape=(224, 224,3)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#第二个卷积层，8个卷积核，每个卷积核大小3*3。
#激活函数用tanh
#采用maxpooling，poolsize为(2,2)
model.add(keras.layers.Convolution2D(8, 3, 3))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Convolution2D(16, 3, 3))
model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#全连接层，先将前一层输出的二维特征图flatten为一维的。
#全连接有128个神经元节点,初始化方式为normal
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
#sigmoid分类，输出是2类别
model.add(keras.layers.Dense(1))
model.add(keras.layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(train_data, train_labels,
         nb_epoch=20, batch_size=30,
         validation_data=(validation_data, validation_labels))

test_loss,test_acc = model.evaluate(validation_data, validation_labels)


Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 3s 2ms/step - loss: 0.6929 - acc: 0.5150 - val_loss: 0.6939 - val_acc: 0.4800
Epoch 2/20
1800/1800 [==============================] - 2s 1ms/step - loss: 0.6917 - acc: 0.5283 - val_loss: 0.6888 - val_acc: 0.6350
Epoch 3/20
1800/1800 [==============================] - 2s 1ms/step - loss: 0.6864 - acc: 0.5578 - val_loss: 0.6859 - val_acc: 0.4950
Epoch 4/20
1800/1800 [==============================] - 2s 1ms/step - loss: 0.6810 - acc: 0.5789 - val_loss: 0.6741 - val_acc: 0.5800
Epoch 5/20
1800/1800 [==============================] - 2s 1ms/step - loss: 0.6689 - acc: 0.5900 - val_loss: 0.6826 - val_acc: 0.5000
Epoch 6/20
1800/1800 [==============================] - 2s 1ms/step - loss: 0.6580 - acc: 0.6256 - val_loss: 0.6448 - val_acc: 0.6500
Epoch 7/20
1800/1800 [==============================] - 2s 1ms/step - loss: 0.6483 - acc: 0.6322 - val_loss: 0.6412 - val_acc: 0.6200
Epoch 8/20
1800

In [69]:
print test_acc

0.69
